In [1]:
# importing numpy and pandas
import numpy as np
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials


# Employee Details

In [2]:
# importing employee data
emp_details = pd.read_excel("emp_details.xlsx",sheet_name="emp")
emp_details['official_In_time' ] = pd.to_datetime(emp_details['official_In_time'],format = '%H:%M:%S')
emp_details['official_out_time' ] = pd.to_datetime(emp_details['official_out_time'],format = '%H:%M:%S')

print(emp_details.info())
emp_details

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   biometric_id       27 non-null     float64       
 1   emp_id             30 non-null     object        
 2   full_name          30 non-null     object        
 3   name               30 non-null     object        
 4   on_service         29 non-null     object        
 5   official_In_time   17 non-null     datetime64[ns]
 6   official_out_time  17 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 1.8+ KB
None


,biometric_id,emp_id,full_name,name,on_service,official_In_time,official_out_time
0,1.0,TIA004,Saheba Khan,Saheba,yes,1900-01-01 11:00:00,1900-01-01 18:30:00
1,3.0,TIA001,Hemant Ranotiya,Hemant,yes,1900-01-01 11:00:00,1900-01-01 20:30:00
2,4.0,TIA009,Abhishek Patel,Abhishek,yes,1900-01-01 11:00:00,1900-01-01 20:30:00
3,5.0,TIA013,Kuldeep Chouhan,Kuldeep,yes,1900-01-01 11:00:00,1900-01-01 18:00:00
4,6.0,Sakshee,Sakshee,Sakshee,yes,1900-01-01 10:00:00,1900-01-01 19:00:00
5,8.0,TIA011,Ajay Gadge,Ajay,no,1900-01-01 09:00:00,1900-01-01 18:00:00
6,9.0,TIA012,Sachin Vaishnave,Sachin,no,1900-01-01 09:00:00,1900-01-01 18:00:00
7,10.0,TIA006,Varun Kumar,Varun,no,1900-01-01 09:30:00,1900-01-01 18:00:00
8,12.0,TIAT005,Bantubala Patidar,Bantubala,yes,1900-01-01 11:30:00,1900-01-01 15:00:00
9,20.0,TIA019,Aditi Chawda,Aditi,yes,1900-01-01 10:00:00,1900-01-01 18:30:00


# Leave Data

In [3]:
# importing leave data - you can just copy and past the leave request data from google sheets directly to the leave sheet as it is

leave_data = pd.read_excel("leave_data.xlsx",sheet_name='leave')
leave_data =  leave_data[['Name','Leave Start Date','Leave End Date','Half Day/ Full Day?','Number of Days',"Type of leave"]]

# creating first name column
leave_data['first name'] = leave_data.apply(lambda x: x["Name"].split(' ')[0].strip(),axis=1)

#data cleaning
leave_data['Leave End Date'] = pd.to_datetime(leave_data['Leave End Date'], errors='coerce')
leave_data['Leave Start Date'] = pd.to_datetime(leave_data['Leave Start Date'], errors='coerce')
leave_data = leave_data[leave_data['Leave Start Date'].notna()]
leave_data.rename(columns={"Name":"full_name"},inplace=True)
leave_data = pd.merge(leave_data,emp_details[["full_name","emp_id"]],how='left',on='full_name')
leave_data.info()
leave_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   full_name            398 non-null    object        
 1   Leave Start Date     398 non-null    datetime64[ns]
 2   Leave End Date       398 non-null    datetime64[ns]
 3   Half Day/ Full Day?  398 non-null    object        
 4   Number of Days       343 non-null    float64       
 5   Type of leave        398 non-null    object        
 6   first name           398 non-null    object        
 7   emp_id               359 non-null    object        
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 25.0+ KB


,full_name,Leave Start Date,Leave End Date,Half Day/ Full Day?,Number of Days,Type of leave,first name,emp_id
0,Anjali Bohra,2022-03-21,2022-03-21,Full day,1.0,Casual Leave,Anjali,Anjali
1,Devendra Patidar,2022-03-24,2022-03-26,Full day,3.0,Sick leave (Illness or Injury),Devendra,Devendra
2,Saheba Khan,2022-03-26,2022-03-26,Full day,1.0,Exam,Saheba,TIA004
3,Saheba Khan,2022-03-29,2022-03-29,Full day,1.0,Exam,Saheba,TIA004
4,Saheba Khan,2022-03-31,2022-03-31,Full day,1.0,Exam,Saheba,TIA004


In [4]:
leave_data

,full_name,Leave Start Date,Leave End Date,Half Day/ Full Day?,Number of Days,Type of leave,first name,emp_id
0,Anjali Bohra,2022-03-21,2022-03-21,Full day,1.0,Casual Leave,Anjali,Anjali
1,Devendra Patidar,2022-03-24,2022-03-26,Full day,3.0,Sick leave (Illness or Injury),Devendra,Devendra
2,Saheba Khan,2022-03-26,2022-03-26,Full day,1.0,Exam,Saheba,TIA004
3,Saheba Khan,2022-03-29,2022-03-29,Full day,1.0,Exam,Saheba,TIA004
4,Saheba Khan,2022-03-31,2022-03-31,Full day,1.0,Exam,Saheba,TIA004
...,...,...,...,...,...,...,...,...
393,Aditi Chawda,2025-05-02,2025-05-02,Full day,NaN,Sick leave (Illness or Injury),Aditi,TIA019
394,Purnima Khandelwal,2025-05-02,2025-05-06,Full day,NaN,Casual Leave,Purnima,TIA024
395,Aditi Chawda,2025-05-07,2025-05-07,Full day,NaN,Work From Home,Aditi,TIA019
396,Purnima Khandelwal,2025-05-07,2025-05-07,Full day,NaN,Casual Leave,Purnima,TIA024


In [5]:
# fixing the structure of leave data table to long format
expanded_data = []
for _, row in leave_data.iterrows():
    start_date = row['Leave Start Date']
    end_date = row['Leave End Date']
    
    # Generate the date range
    dates = pd.date_range(start=start_date, end=end_date)
    
    for date in dates:
        expanded_data.append({
            'full_name': row['full_name'],
            "first name" : row["first name"],
            "emp_id":row["emp_id"],
            'Date': date,
            'Half Day/ Full Day?': row['Half Day/ Full Day?'],
            "Type of leave" : row['Type of leave']
        })
# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_data)
leave_data = expanded_df
leave_data.info()
leave_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   full_name            701 non-null    object        
 1   first name           701 non-null    object        
 2   emp_id               579 non-null    object        
 3   Date                 701 non-null    datetime64[ns]
 4   Half Day/ Full Day?  701 non-null    object        
 5   Type of leave        701 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 33.0+ KB


,full_name,first name,emp_id,Date,Half Day/ Full Day?,Type of leave
0,Anjali Bohra,Anjali,Anjali,2022-03-21,Full day,Casual Leave
1,Devendra Patidar,Devendra,Devendra,2022-03-24,Full day,Sick leave (Illness or Injury)
2,Devendra Patidar,Devendra,Devendra,2022-03-25,Full day,Sick leave (Illness or Injury)
3,Devendra Patidar,Devendra,Devendra,2022-03-26,Full day,Sick leave (Illness or Injury)
4,Saheba Khan,Saheba,TIA004,2022-03-26,Full day,Exam


In [6]:
# calculating and creating columns for fullday leave, halfday leave or workfrom home

leave_data['is on full day leave'] = np.where(leave_data['Half Day/ Full Day?']=='Full day',1,0)
leave_data['is on half day leave'] = np.where(leave_data['Half Day/ Full Day?']!="Full day",1,0)
leave_data['is work from home'] = np.where(leave_data['Type of leave']=="Work From Home",1,0)
leave_data['is on full day leave'] = leave_data.apply(lambda x : 0 if x["is work from home"]==1 else x["is on full day leave"] ,axis=1)

leave_data.to_excel("leaves.xlsx",index=False)
leave_data.head()

,full_name,first name,emp_id,Date,Half Day/ Full Day?,Type of leave,is on full day leave,is on half day leave,is work from home
0,Anjali Bohra,Anjali,Anjali,2022-03-21,Full day,Casual Leave,1,0,0
1,Devendra Patidar,Devendra,Devendra,2022-03-24,Full day,Sick leave (Illness or Injury),1,0,0
2,Devendra Patidar,Devendra,Devendra,2022-03-25,Full day,Sick leave (Illness or Injury),1,0,0
3,Devendra Patidar,Devendra,Devendra,2022-03-26,Full day,Sick leave (Illness or Injury),1,0,0
4,Saheba Khan,Saheba,TIA004,2022-03-26,Full day,Exam,1,0,0


In [7]:
# creating a unique ID columns
leave_data["unique_id"] = leave_data['Date'].astype(str) +"-"+ leave_data['emp_id']

In [8]:
leave_data.head()

,full_name,first name,emp_id,Date,Half Day/ Full Day?,Type of leave,is on full day leave,is on half day leave,is work from home,unique_id
0,Anjali Bohra,Anjali,Anjali,2022-03-21,Full day,Casual Leave,1,0,0,2022-03-21-Anjali
1,Devendra Patidar,Devendra,Devendra,2022-03-24,Full day,Sick leave (Illness or Injury),1,0,0,2022-03-24-Devendra
2,Devendra Patidar,Devendra,Devendra,2022-03-25,Full day,Sick leave (Illness or Injury),1,0,0,2022-03-25-Devendra
3,Devendra Patidar,Devendra,Devendra,2022-03-26,Full day,Sick leave (Illness or Injury),1,0,0,2022-03-26-Devendra
4,Saheba Khan,Saheba,TIA004,2022-03-26,Full day,Exam,1,0,0,2022-03-26-TIA004


# Holiday Calendar Data

In [9]:
# importing the information for holidays
holidays = pd.read_excel("holidays.xlsx")
holidays.info()
holidays.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     16 non-null     datetime64[ns]
 1   holiday  16 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 388.0+ bytes


,date,holiday
0,2025-01-01,New Year
1,2025-01-14,Makar Sankranti
2,2025-01-26,Republic Day
3,2025-03-14,Holi
4,2025-03-19,Rang Panchmi


# Attendance Data

## Importing and cleaning attendance data

In [10]:
# this is the main file which has the attendance data and reading it in csv
df = pd.read_csv("attendance.dat",header=None)
def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

df["id"]= df[0].map(get_id)
df["punch_time"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)   # deleting the raw data column and keeping only two column
df['id']= df["id"].astype(int)      # conerting id column to int
df['punch_time'] = pd.to_datetime(df['punch_time'])
df['punch_date'] = df['punch_time'].dt.date
df['punch_timeonly']= df['punch_time'].dt.time
df['punch_date'] = pd.to_datetime(df['punch_date'])
df['punch_timeonly'] = pd.to_datetime(df['punch_timeonly'],format = '%H:%M:%S')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              519 non-null    int64         
 1   punch_time      519 non-null    datetime64[ns]
 2   punch_date      519 non-null    datetime64[ns]
 3   punch_timeonly  519 non-null    datetime64[ns]
dtypes: datetime64[ns](3), int64(1)
memory usage: 16.3 KB


,id,punch_time,punch_date,punch_timeonly
0,23,2025-04-01 09:17:01,2025-04-01,1900-01-01 09:17:01
1,25,2025-04-01 09:25:18,2025-04-01,1900-01-01 09:25:18
2,28,2025-04-01 09:57:22,2025-04-01,1900-01-01 09:57:22
3,24,2025-04-01 09:59:49,2025-04-01,1900-01-01 09:59:49
4,1,2025-04-01 10:50:40,2025-04-01,1900-01-01 10:50:40


## Prepare attendance data

In [11]:
df.sort_values(by=['id','punch_time'],inplace=True)
df_intime = pd.DataFrame(df.groupby(['id','punch_date']).min()['punch_timeonly']).reset_index()
df_outtime = pd.DataFrame(df.groupby(['id','punch_date']).max()['punch_timeonly']).reset_index()
df_intime.rename(columns={'punch_timeonly':'punch_intime'},inplace=True)
df_outtime.rename(columns={'punch_timeonly':'punch_outtime'},inplace=True)
df = pd.merge(df_intime,df_outtime,on=['id','punch_date'],how='left')
df.head()

,id,punch_date,punch_intime,punch_outtime
0,1,2025-04-01,1900-01-01 10:50:40,1900-01-01 19:02:35
1,1,2025-04-02,1900-01-01 10:36:48,1900-01-01 19:09:44
2,1,2025-04-03,1900-01-01 10:23:39,1900-01-01 10:23:39
3,1,2025-04-04,1900-01-01 10:40:03,1900-01-01 18:56:18
4,1,2025-04-05,1900-01-01 10:40:27,1900-01-01 19:18:28


# All dates and employee combinantion

## creating all dates data

In [12]:
# finding the first day and last day of month
a = df['punch_date'][0]
first_day = a.replace(day=1).normalize()
last_day = a + pd.offsets.MonthEnd(0)

In [13]:
# creating date table with all dates from the month with there week day name
all_dates_in_data = pd.date_range(start=first_day,end=last_day)
all_dates_in_data = pd.to_datetime(all_dates_in_data)
all_dates_in_data = pd.DataFrame(pd.DataFrame(all_dates_in_data)[0].dt.date)
all_dates_in_data['date'] = pd.to_datetime(all_dates_in_data[0])
all_dates_in_data['week name'] = all_dates_in_data['date'].dt.strftime("%a")
all_dates_in_data.drop(columns=0,inplace=True)
print(all_dates_in_data.info())
all_dates_in_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       30 non-null     datetime64[ns]
 1   week name  30 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 612.0+ bytes
None


,date,week name
0,2025-04-01,Tue
1,2025-04-02,Wed
2,2025-04-03,Thu
3,2025-04-04,Fri
4,2025-04-05,Sat


## creating a list of all employees in the data

In [14]:
# getting the id of all those pople who are actively coming in company who are on service

all_employee_names = pd.DataFrame(emp_details[emp_details['biometric_id'].apply( lambda x : x in list(df['id'].unique()))]['emp_id'],columns=["emp_id"])
all_employee_names

,emp_id
0,TIA004
1,TIA001
2,TIA009
3,TIA013
4,Sakshee
7,TIA006
8,TIAT005
9,TIA019
10,TIA023
11,TIA024


## cross join between both dates and employee table 

In [15]:
all_emp_and_date_combination = pd.merge(all_dates_in_data,all_employee_names,how="cross")

In [16]:
# creating a unique id here also for better join between emp and leave data
all_emp_and_date_combination['unique_id'] = all_emp_and_date_combination['date'].astype(str) +"-"+all_emp_and_date_combination['emp_id']

In [17]:
all_emp_and_date_combination.head()

,date,week name,emp_id,unique_id
0,2025-04-01,Tue,TIA004,2025-04-01-TIA004
1,2025-04-01,Tue,TIA001,2025-04-01-TIA001
2,2025-04-01,Tue,TIA009,2025-04-01-TIA009
3,2025-04-01,Tue,TIA013,2025-04-01-TIA013
4,2025-04-01,Tue,Sakshee,2025-04-01-Sakshee


# Final data

In [18]:
# merge emp details with df
df = pd.merge(df,emp_details,how='left', left_on='id',right_on='biometric_id')

In [19]:
# creating unique id for better joins
df['unique_id'] = df['punch_date'].astype(str) + "-" + df['emp_id']

In [20]:
# alall_emp_and_date_combination with df
df = pd.merge(all_emp_and_date_combination,df,how="left",on='unique_id')

In [21]:
# then df with holiday 
df = pd.merge(df,holidays,how="left",on='date')

In [22]:
# merging leave data
df = pd.merge(df,leave_data[['full_name','Date','is on full day leave',	'is on half day leave',	'is work from home','unique_id']],how="left",on='unique_id')

In [23]:
# calculating late or early leave
df['late_arival'] = np.where(df['punch_intime']> ( df['official_In_time']+pd.to_timedelta("10m")),1,0)
df['early_leave'] = np.where(df['punch_outtime']<(df['official_out_time'] - pd.to_timedelta("10m")),1,0)

In [24]:
df.rename(columns={'emp_id_x':"emp_id"},inplace=True)

In [25]:
df = pd.merge(df,emp_details[['emp_id','full_name','biometric_id']],how='left',on='emp_id')

In [26]:
df = df[['date',
           'emp_id',
           'biometric_id_y',
             'full_name',
           'official_In_time',
            'official_out_time',
           'punch_intime',
            'punch_outtime',
           'week name',
           'holiday',
           'is on full day leave',
            'is on half day leave',
            'is work from home',
            'late_arival', 
           'early_leave']]    

In [27]:
df.rename(columns={'biometric_id_y':"biometric_id"},inplace=True)

In [28]:
# final data
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  390 non-null    datetime64[ns]
 1   emp_id                390 non-null    object        
 2   biometric_id          390 non-null    float64       
 3   full_name             390 non-null    object        
 4   official_In_time      270 non-null    datetime64[ns]
 5   official_out_time     270 non-null    datetime64[ns]
 6   punch_intime          270 non-null    datetime64[ns]
 7   punch_outtime         270 non-null    datetime64[ns]
 8   week name             390 non-null    object        
 9   holiday               13 non-null     object        
 10  is on full day leave  16 non-null     float64       
 11  is on half day leave  16 non-null     float64       
 12  is work from home     16 non-null     float64       
 13  late_arival         

,date,emp_id,biometric_id,full_name,official_In_time,official_out_time,punch_intime,punch_outtime,week name,holiday,is on full day leave,is on half day leave,is work from home,late_arival,early_leave
0,2025-04-01,TIA004,1.0,Saheba Khan,1900-01-01 11:00:00,1900-01-01 18:30:00,1900-01-01 10:50:40,1900-01-01 19:02:35,Tue,NaN,NaN,NaN,NaN,0,0
1,2025-04-01,TIA001,3.0,Hemant Ranotiya,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0
2,2025-04-01,TIA009,4.0,Abhishek Patel,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0
3,2025-04-01,TIA013,5.0,Kuldeep Chouhan,1900-01-01 11:00:00,1900-01-01 18:00:00,1900-01-01 10:58:09,1900-01-01 18:53:30,Tue,NaN,NaN,NaN,NaN,0,0
4,2025-04-01,Sakshee,6.0,Sakshee,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0


In [29]:
# appending the data which we clean to the all data file
try :
    filedf = pd.read_excel("main_data.xlsx")
except:
    pd.DataFrame().to_excel("main_data.xlsx")
    filedf = pd.read_excel("main_data.xlsx")
filedf = pd.concat((filedf,df))
filedf = filedf.drop_duplicates()
filedf.to_excel("main_data.xlsx",index=False)

# Rough

In [31]:
start = pd.to_datetime("1900-1-1")

In [74]:
pd.to_timedelta('10:12:38').total_seconds()

36758.0

In [71]:
delta = pd.to_timedelta('12:00:00')  # 12 hours

# Convert to Excel number format
excel_time = delta.total_seconds() / 86400

print(excel_time)  # Output: 0.5 (since 12 hours is half a day)

0.5


In [75]:
df.head()

,date,emp_id,biometric_id,full_name,official_In_time,official_out_time,punch_intime,punch_outtime,week name,holiday,is on full day leave,is on half day leave,is work from home,late_arival,early_leave,days in number
0,2025-04-01,TIA004,1.0,Saheba Khan,1900-01-01 11:00:00,1900-01-01 18:30:00,1900-01-01 10:50:40,1900-01-01 19:02:35,Tue,NaN,NaN,NaN,NaN,0,0,45746
1,2025-04-01,TIA001,3.0,Hemant Ranotiya,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0,45746
2,2025-04-01,TIA009,4.0,Abhishek Patel,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0,45746
3,2025-04-01,TIA013,5.0,Kuldeep Chouhan,1900-01-01 11:00:00,1900-01-01 18:00:00,1900-01-01 10:58:09,1900-01-01 18:53:30,Tue,NaN,NaN,NaN,NaN,0,0,45746
4,2025-04-01,Sakshee,6.0,Sakshee,NaT,NaT,NaT,NaT,Tue,NaN,NaN,NaN,NaN,0,0,45746


In [80]:
df['gg'] = (df['punch_intime'] - pd.to_datetime("1900-1-1"))

In [84]:
df['gg'].apply(lambda x : x.total_seconds() / 86400)

0      0.451852
1           NaN
2           NaN
3      0.457049
4           NaN
         ...   
385    0.391701
386    0.579525
387    0.387141
388    0.348935
389         NaN
Name: gg, Length: 390, dtype: float64

In [96]:
df.drop(columns='gg',inplace=True)

In [86]:
## code to convert datetime into excel format time in decimal

(df['punch_intime'] - pd.to_datetime("1900-1-1")).apply(lambda x : x.total_seconds() / 86400)

0      0.451852
1           NaN
2           NaN
3      0.457049
4           NaN
         ...   
385    0.391701
386    0.579525
387    0.387141
388    0.348935
389         NaN
Name: punch_intime, Length: 390, dtype: float64

### converting datetime into excel format time

In [87]:
df['punch_intime'] = (df['punch_intime'] - pd.to_datetime("1900-1-1")).apply(lambda x : x.total_seconds() / 86400)

In [88]:
df['official_In_time'] = (df['official_In_time'] - pd.to_datetime("1900-1-1")).apply(lambda x : x.total_seconds() / 86400)

In [89]:
df['official_out_time'] = (df['official_out_time'] - pd.to_datetime("1900-1-1")).apply(lambda x : x.total_seconds() / 86400)

In [90]:
df['punch_outtime'] = (df['punch_outtime'] - pd.to_datetime("1900-1-1")).apply(lambda x : x.total_seconds() / 86400)

In [93]:
df['date'] = (df['date'] - start).astype(str).apply(lambda x : int(x.split()[0]))

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

In [97]:
df.head()

,date,emp_id,biometric_id,full_name,official_In_time,official_out_time,punch_intime,punch_outtime,week name,holiday,is on full day leave,is on half day leave,is work from home,late_arival,early_leave,days in number
0,45746,TIA004,1.0,Saheba Khan,0.458333,0.770833,0.451852,0.793461,Tue,NaN,NaN,NaN,NaN,0,0,45746
1,45746,TIA001,3.0,Hemant Ranotiya,NaN,NaN,NaN,NaN,Tue,NaN,NaN,NaN,NaN,0,0,45746
2,45746,TIA009,4.0,Abhishek Patel,NaN,NaN,NaN,NaN,Tue,NaN,NaN,NaN,NaN,0,0,45746
3,45746,TIA013,5.0,Kuldeep Chouhan,0.458333,0.750000,0.457049,0.787153,Tue,NaN,NaN,NaN,NaN,0,0,45746
4,45746,Sakshee,6.0,Sakshee,NaN,NaN,NaN,NaN,Tue,NaN,NaN,NaN,NaN,0,0,45746


In [98]:

# Define the scopes for Google Sheets and Drive
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Load credentials from JSON file
creds = Credentials.from_service_account_file("iota-attendance-462806-b57e98e44b34.json", scopes=SCOPES)

# Authorize gspread
client = gspread.authorize(creds)

# Open the Google Sheet by file name
spreadsheet = client.open("iota")  # Replace with the document title

# List all worksheet/tab names
sheets = spreadsheet.worksheets()

print("Available tabs:")
for sheet in sheets:
    print("-", sheet.title)

# Select the first tab (or any by name)
worksheet = sheets[0]  # or use spreadsheet.worksheet("Exact Tab Name")

# Get all records
# data = worksheet.get_all_records()

# Show results
# print(data)


Available tabs:
- att


In [106]:
dataf =  df.values.tolist()

In [110]:
# this is to update the df in sheet

worksheet.clear()
dataf = [df.columns.values.tolist()] + df.values.tolist()
worksheet.update("A1", dataf)

C:\Users\deves\AppData\Local\Temp\ipykernel_7740\1993220901.py:5: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A1", dataf)


{'spreadsheetId': '14QJgYt8jCA2p0L7wdsEDtqwqspYTg5InY7rPJlKZxQU',
 'updatedRange': 'att!A1:P391',
 'updatedRows': 391,
 'updatedColumns': 16,
 'updatedCells': 6256}

In [114]:
existing_data = len(sheet.get_all_values())
worksheet.update("A"+str(existing_data+1), df.values.tolist())

C:\Users\deves\AppData\Local\Temp\ipykernel_7740\1212046108.py:2: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A"+str(existing_data+1), df.values.tolist())


{'spreadsheetId': '14QJgYt8jCA2p0L7wdsEDtqwqspYTg5InY7rPJlKZxQU',
 'updatedRange': 'att!A392:P781',
 'updatedRows': 390,
 'updatedColumns': 16,
 'updatedCells': 6240}

In [112]:
existing_data

391

In [115]:
for i in df.values.tolist():
    print(i)
    break

[45746, 'TIA004', 1.0, 'Saheba Khan', 0.4583333333333333, 0.7708333333333334, 0.45185185185185184, 0.7934606481481481, 'Tue', 0, 0.0, 0.0, 0.0, 0, 0, 45746]


In [117]:
sheet.get_all_values().extend(df.values.tolist())

In [102]:
df.fillna(0,inplace=True)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  390 non-null    int64  
 1   emp_id                390 non-null    object 
 2   biometric_id          390 non-null    float64
 3   full_name             390 non-null    object 
 4   official_In_time      390 non-null    float64
 5   official_out_time     390 non-null    float64
 6   punch_intime          390 non-null    float64
 7   punch_outtime         390 non-null    float64
 8   week name             390 non-null    object 
 9   holiday               390 non-null    object 
 10  is on full day leave  390 non-null    float64
 11  is on half day leave  390 non-null    float64
 12  is work from home     390 non-null    float64
 13  late_arival           390 non-null    int64  
 14  early_leave           390 non-null    int64  
 15  days in number        3

In [110]:
df[.columns]

Index(['date', 'emp_id', 'biometric_id', 'full_name', 'official_In_time',
       'official_out_time', 'punch_intime', 'punch_outtime', 'week name',
       'holiday', 'is on full day leave', 'is on half day leave',
       'is work from home', 'late_arival', 'early_leave'],
      dtype='object')

In [93]:
df['date'] = df['date'].astype("str")

In [94]:
df['official_In_time'] = df['official_In_time'].astype("str")

In [95]:
df['official_out_time'] = df['official_out_time'].astype("str")

In [96]:
df['punch_intime'] = df['punch_intime'].astype("str")

In [97]:
df['punch_outtime'] = df['punch_outtime'].astype("str")

In [98]:
df['biometric_id'] = df['biometric_id'].astype("str")

In [99]:
df['is on full day leave'] = df['is on full day leave'].astype("str")

In [100]:
df['is on half day leave'] = df['is on half day leave'].astype("str")

In [101]:
df['is work from home'] = df['is work from home'].astype("str")

In [102]:
df['late_arival'] = df['late_arival'].astype("str")

In [103]:
df['early_leave'] = df['early_leave'].astype("str")

In [119]:
df['holiday'].fillna("null",inplace=True)

C:\Users\deves\AppData\Local\Temp\ipykernel_12684\1669106505.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['holiday'].fillna("null",inplace=True)


In [104]:
df.head()

,date,emp_id,biometric_id,full_name,official_In_time,official_out_time,punch_intime,punch_outtime,week name,holiday,is on full day leave,is on half day leave,is work from home,late_arival,early_leave
0,2025-04-01,TIA004,1.0,Saheba Khan,1900-01-01 11:00:00,1900-01-01 18:30:00,1900-01-01 10:50:40,1900-01-01 19:02:35,Tue,NaN,nan,nan,nan,0,0
1,2025-04-01,TIA001,3.0,Hemant Ranotiya,NaT,NaT,NaT,NaT,Tue,NaN,nan,nan,nan,0,0
2,2025-04-01,TIA009,4.0,Abhishek Patel,NaT,NaT,NaT,NaT,Tue,NaN,nan,nan,nan,0,0
3,2025-04-01,TIA013,5.0,Kuldeep Chouhan,1900-01-01 11:00:00,1900-01-01 18:00:00,1900-01-01 10:58:09,1900-01-01 18:53:30,Tue,NaN,nan,nan,nan,0,0
4,2025-04-01,Sakshee,6.0,Sakshee,NaT,NaT,NaT,NaT,Tue,NaN,nan,nan,nan,0,0


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  390 non-null    object
 1   emp_id                390 non-null    object
 2   biometric_id          390 non-null    object
 3   full_name             390 non-null    object
 4   official_In_time      390 non-null    object
 5   official_out_time     390 non-null    object
 6   punch_intime          390 non-null    object
 7   punch_outtime         390 non-null    object
 8   week name             390 non-null    object
 9   holiday               390 non-null    object
 10  is on full day leave  390 non-null    object
 11  is on half day leave  390 non-null    object
 12  is work from home     390 non-null    object
 13  late_arival           390 non-null    object
 14  early_leave           390 non-null    object
dtypes: object(15)
memory usage: 45.8+ KB
